In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
import smtplib
import ssl
from email.message import EmailMessage
import threading
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.mixer.init()
pygame.mixer.music.load("alarm.wav")

In [3]:
model = YOLO("prod_deployment_path/best.pt")


In [4]:
def send_email_alert(image):
    msg = EmailMessage()
    msg['Subject'] = "Driver Drowsiness Alert!"
    msg['From'] = "doddasowmika0@gmail.com"          # Change this
    msg['To'] = "doddashilpika@gmail.com"       # Change this
    msg.set_content("Warning! The driver appears drowsy. Please check immediately.")

    # Encode image as JPEG and attach
    _, img_encoded = cv2.imencode('.jpg', image)
    img_bytes = img_encoded.tobytes()
    msg.add_attachment(img_bytes, maintype='image', subtype='jpeg', filename='drowsy_alert.jpg')

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as server:
        server.login("doddasowmika0@gmail.com", "ltnb trtg bwti hdnk")  # Use app password or real password carefully
        server.send_message(msg)
    print("Email alert sent!")

In [5]:
alert_active = False
def alert_driver(frame):
    global alert_active
    if not alert_active:
        alert_active = True
        pygame.mixer.music.play(-1)  # Play alarm sound loop
        threading.Thread(target=send_email_alert, args=(frame,)).start()

In [ ]:
def stop_alert():
    global alert_active
    pygame.mixer.music.stop()
    alert_active = False

# Video capture (0 for default webcam)
cap = cv2.VideoCapture(0)

drowsy_frame_count = 0
drowsy_threshold = 15  # number of consecutive frames detecting drowsy to trigger alert

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Run inference
    results = model(frame)[0]

    # Default state
    driver_state = "Awake"

    # Process detected objects
    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, score, cls = box
        label = model.names[int(cls)]

        # Draw bounding box and label
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, label, (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        if label.lower() == "drowsy":
            driver_state = "Drowsy"

    # Drowsiness logic
    if driver_state == "Drowsy":
        drowsy_frame_count += 1
    else:
        drowsy_frame_count = 0
        stop_alert()

    if drowsy_frame_count > drowsy_threshold:
        alert_driver(frame)
        cv2.putText(frame, "DROWSINESS ALERT!", (50, 50),
                    cv2.FONT_HERSHEY_DUPLEX, 1.5, (0, 0, 255), 3)
    else:
        cv2.putText(frame, "Driver is Awake", (50, 50),
                    cv2.FONT_HERSHEY_DUPLEX, 1.5, (0, 255, 0), 3)

    # Show the frame
    cv2.imshow("Driver Drowsiness Detection", frame)

    # Exit on pressing ESC key
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
stop_alert()


0: 480x640 (no detections), 144.9ms
Speed: 17.6ms preprocess, 144.9ms inference, 5.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 awakes, 1 drowsy, 83.6ms
Speed: 2.7ms preprocess, 83.6ms inference, 5.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 awakes, 1 drowsy, 119.1ms
Speed: 10.4ms preprocess, 119.1ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 awakes, 89.4ms
Speed: 2.5ms preprocess, 89.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 awakes, 80.1ms
Speed: 1.7ms preprocess, 80.1ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 awakes, 1 drowsy, 81.2ms
Speed: 1.6ms preprocess, 81.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 awakes, 1 drowsy, 75.3ms
Speed: 1.6ms preprocess, 75.3ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 awakes, 1 drowsy, 75.5ms
Speed: 2.1ms preprocess, 7

Exception in thread Thread-5 (send_email_alert):
Traceback (most recent call last):
  File "C:\Users\Hp\.conda\New folder\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Hp\.conda\New folder\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_8256\1321863988.py", line 14, in send_email_alert
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 1050, in __init__
    SMTP.__init__(self, host, port, local_hostname, timeout,
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 255, in __init__
    (code, msg) = self.connect(host, port)
                  ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 341, in connect
    self.sock = self._get_socket(host, port, self.timeout)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 1056, in _get_socket
    new_socket = 

0: 480x640 1 awake, 1 drowsy, 89.6ms
Speed: 1.9ms preprocess, 89.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 66.3ms
Speed: 1.7ms preprocess, 66.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 2 drowsys, 65.4ms
Speed: 2.4ms preprocess, 65.4ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 66.8ms
Speed: 3.7ms preprocess, 66.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 drowsys, 65.6ms
Speed: 2.0ms preprocess, 65.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 66.5ms
Speed: 2.7ms preprocess, 66.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 2 drowsys, 66.2ms
Speed: 4.4ms preprocess, 66.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 70.3ms
Speed: 2.0ms preprocess,

Exception in thread Thread-6 (send_email_alert):
Traceback (most recent call last):
  File "C:\Users\Hp\.conda\New folder\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Hp\.conda\New folder\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_8256\1321863988.py", line 14, in send_email_alert
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 1050, in __init__
    SMTP.__init__(self, host, port, local_hostname, timeout,
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 255, in __init__
    (code, msg) = self.connect(host, port)
                  ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 341, in connect
    self.sock = self._get_socket(host, port, self.timeout)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 1056, in _get_socket
    new_socket = 

0: 480x640 1 awake, 1 drowsy, 78.3ms
Speed: 1.7ms preprocess, 78.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 67.8ms
Speed: 1.6ms preprocess, 67.8ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 68.6ms
Speed: 2.1ms preprocess, 68.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 drowsys, 65.2ms
Speed: 2.1ms preprocess, 65.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 drowsys, 66.2ms
Speed: 2.7ms preprocess, 66.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 2 drowsys, 65.2ms
Speed: 2.8ms preprocess, 65.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 drowsys, 65.0ms
Speed: 2.2ms preprocess, 65.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 drowsys, 71.4ms
Speed: 2.1ms preprocess, 71.4ms inference, 1.0ms 

...

Exception in thread Thread-13 (send_email_alert):
Traceback (most recent call last):
  File "C:\Users\Hp\.conda\New folder\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Hp\.conda\New folder\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_8256\1321863988.py", line 14, in send_email_alert
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 1050, in __init__
    SMTP.__init__(self, host, port, local_hostname, timeout,
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 255, in __init__
    (code, msg) = self.connect(host, port)
                  ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 341, in connect
    self.sock = self._get_socket(host, port, self.timeout)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 1056, in _get_socket
    new_socket =

0: 480x640 1 awake, 1 drowsy, 86.4ms
Speed: 4.6ms preprocess, 86.4ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 69.2ms
Speed: 1.8ms preprocess, 69.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 2 drowsys, 87.8ms
Speed: 2.2ms preprocess, 87.8ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 80.7ms
Speed: 1.9ms preprocess, 80.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 74.4ms
Speed: 2.5ms preprocess, 74.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 79.2ms
Speed: 2.5ms preprocess, 79.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 73.4ms
Speed: 1.9ms preprocess, 73.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 71.7ms
Speed: 2.5ms prep


0: 480x640 1 awake, 1 drowsy, 83.8ms
Speed: 2.2ms preprocess, 83.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 88.4ms
Speed: 1.4ms preprocess, 88.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 66.2ms
Speed: 1.4ms preprocess, 66.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 64.8ms
Speed: 1.6ms preprocess, 64.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 66.2ms
Speed: 3.6ms preprocess, 66.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 65.4ms
Speed: 2.4ms preprocess, 65.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 64.7ms
Speed: 2.4ms preprocess, 64.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 69.7ms
Speed: 1.9ms prep

Exception in thread Thread-14 (send_email_alert):
Traceback (most recent call last):
  File "C:\Users\Hp\.conda\New folder\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Hp\.conda\New folder\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_8256\1321863988.py", line 14, in send_email_alert
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 1050, in __init__
    SMTP.__init__(self, host, port, local_hostname, timeout,
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 255, in __init__
    (code, msg) = self.connect(host, port)
                  ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 341, in connect
    self.sock = self._get_socket(host, port, self.timeout)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\.conda\New folder\Lib\smtplib.py", line 1056, in _get_socket
    new_socket =

0: 480x640 2 drowsys, 74.6ms
Speed: 1.9ms preprocess, 74.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 drowsys, 74.6ms
Speed: 2.0ms preprocess, 74.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 drowsy, 90.5ms
Speed: 1.5ms preprocess, 90.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 drowsy, 68.5ms
Speed: 2.1ms preprocess, 68.5ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 drowsy, 63.3ms
Speed: 1.8ms preprocess, 63.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 64.8ms
Speed: 2.9ms preprocess, 64.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 awake, 1 drowsy, 66.3ms
Speed: 2.3ms preprocess, 66.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 drowsy, 64.2ms
Speed: 3.1ms preprocess, 64.2ms inference, 1.0ms postprocess per image